In [ ]:
import numpy as np

from dune.xt.common import init_logger, init_mpi
init_mpi()
init_logger()

In [ ]:
print('init grid and problem ...')

from dune.xt.grid import (
    make_cube_grid__2d_simplex_aluconform as make_cube_grid,
    make_boundary_info_on_leaf_layer as make_boundary_info,
    make_apply_on_dirichlet_intersections_leaf_part as make_apply_on_dirichlet_intersections,
    make_apply_on_neumann_intersections_leaf_part as make_apply_on_neumann_intersections
)

grid = make_cube_grid(lower_left=[-1, -1], upper_right=[1, 1], num_elements=[4, 4], num_refinements=2)
boundary_info = make_boundary_info(grid, 'xt.grid.boundaryinfo.alldirichlet')
apply_on_dirichlet_intersections = make_apply_on_dirichlet_intersections(boundary_info)
apply_on_neumann_intersections = make_apply_on_neumann_intersections(boundary_info)

In [ ]:
from dune.xt.functions import (
    make_constant_function_1x1,
    make_constant_function_2x2,
    make_expression_function_1x1
)

lambda_ = make_constant_function_1x1(grid, 1.0, name='lambda')
kappa = make_constant_function_2x2(grid, [[1., 0.], [0., 1.]], name='kappa')
f = make_expression_function_1x1(grid, 'x', '0.5*pi*pi*cos(0.5*pi*x[0])*cos(0.5*pi*x[1])', order=2, name='f')
g_D = make_constant_function_1x1(grid, 0.0, name='g_D')
g_N = make_constant_function_1x1(grid, 0.0, name='g_N')

In [ ]:
print('discretizing ...')

from dune.xt.la import (
    IstlRowMajorSparseMatrixDouble as Matrix,
    IstlDenseVectorDouble as Vector,
    make_solver
)

from dune.gdt import (
    make_dg_leaf_part_to_1x1_fem_p1_space as make_dg_space,
    make_elliptic_swipdg_matrix_operator,
    make_l2_volume_vector_functional,
    make_l2_face_vector_functional,
    make_elliptic_swipdg_dirichlet_vector_functional,
    make_system_assembler,
    make_discrete_function
)

space = make_dg_space(grid)

system_matrix = Matrix(space.size(), space.size(), space.compute_pattern('face_and_volume'))
swipdg_operator = make_elliptic_swipdg_matrix_operator(lambda_, kappa, boundary_info, system_matrix, space)

rhs_vector = Vector(space.size(), 0.0)
l2_force_functional = make_l2_volume_vector_functional(f, rhs_vector, space)
swipdg_dirichlet_functional = make_elliptic_swipdg_dirichlet_vector_functional(g_D, lambda_, kappa,
                                                                               boundary_info, rhs_vector,
                                                                               space)
l2_neumann_functional = make_l2_face_vector_functional(g_N, rhs_vector, space)

system_assembler = make_system_assembler(space)
system_assembler.append(swipdg_operator)
system_assembler.append(l2_neumann_functional, apply_on_neumann_intersections)
system_assembler.append(l2_force_functional)
system_assembler.append(swipdg_dirichlet_functional)
system_assembler.assemble()

u_h_vector = Vector(space.size(), 0.0)
make_solver(system_matrix).apply(rhs_vector, u_h_vector)
u_h = make_discrete_function(space, u_h_vector, 'u_h')
u_h.visualize('../u_h')

In [ ]:
print('estimating error ...')
print('')

from dune.xt.functions import make_esv2007_cutoff_function_diffusion_factor_and_tensor_to_1x1 as make_esv2007_cutoff_function

from dune.gdt import (
    make_elliptic_matrix_operator_istl_row_major_sparse_matrix_double as make_elliptic_matrix_operator,
    apply_oswald_interpolation_operator,
    make_rt_leaf_view_to_2x1_pdelab_p0_space as make_rt_space,
    #make_fv_leaf_view_to_1x1_gdt_space as make_fv_space,
    #project,
    apply_diffusive_flux_reconstruction_operator,
    apply_weighted_l2_product_leaf_view as apply_weighted_l2_product,
    make_local_diffusive_flux_estimation_esv2007_operator_to_1x1,
    ResultStorage
)

elliptic_operator_mu_bar = make_elliptic_matrix_operator(lambda_, kappa, space)

# the nonconformity indicator
u_h_in_H10_vector = Vector(space.size(), 0.0)
u_h_in_H10 = make_discrete_function(space, u_h_in_H10_vector, 'u_h_in_H10')
apply_oswald_interpolation_operator(u_h, u_h_in_H10)
#u_h_in_H10.visualize('../u_h_in_H10')
#make_discrete_function(space, u_h_vector - u_h_in_H10_vector, 'u_h - u_h_in_H10').visualize('../u_h__minus__u_h_in_H10')
eta_nc = elliptic_operator_mu_bar.induced_norm(u_h_vector - u_h_in_H10_vector)
print('  nonconformity indicator: {}'.format(eta_nc)) # should be 1.66e-01

# the residual indicator
rt_space = make_rt_space(grid)
flux_reconxtruction_vector = Vector(rt_space.size(), 0.0)
flux_reconstruction = make_discrete_function(rt_space, flux_reconxtruction_vector, 'flux reconstruction')
apply_diffusive_flux_reconstruction_operator(lambda_, kappa, u_h, flux_reconstruction)
#flux_reconstruction.visualize('../flux_reconstruction')
flux_divergence = flux_reconstruction.divergence()
#flux_divergence.visualize(grid, '../flux_divergence')
#(f - flux_divergence).visualize(grid, '../f_minus_flux_divergence')
# side note: the divergence of the flix should coincide with the P0-projection of f
#fv_space = make_fv_space(grid)
#f_h_vector = Vector(fv_space.size(), 0.)
#f_h = make_discrete_function(fv_space, f_h_vector, 'f_h')
#project(f, f_h)
#f_h.visualize('f_h')
#(f_h - flux_divergence).visualize(grid, '../f_h_minus_flux_divergence')
cutoff_function = make_esv2007_cutoff_function(grid, lambda_, kappa, name='Cutoff function')
#cutoff_function.visualize(grid, '../cutoff_function')
eta_r = np.sqrt(apply_weighted_l2_product(cutoff_function, f - flux_divergence, f - flux_divergence, grid, level=-1))
print('  residual indicator: {}'.format(eta_r)) # should be 7.23e-02

# the diffusive flux estimator
local_diffusive_flux_estimation_operator = make_local_diffusive_flux_estimation_esv2007_operator_to_1x1(lambda_, kappa, flux_reconstruction, over_integrate=2)
eta_df = ResultStorage()
system_assembler.append(local_diffusive_flux_estimation_operator, u_h, u_h, eta_df)
system_assembler.assemble()
eta_df = np.sqrt(eta_df.result)
print('  diffusive flux indicator: {}'.format(eta_df)) # should be 3.55e-01

print('')
print('  estimated error: {}'.format(np.sqrt(eta_nc + eta_r + eta_df))) # should be 7.70e-01